# Big Data Modeling and Management Assigment (Question 9 Revision)


## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_ 

#### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions


#### Questions


0. __Example Question__ _How many beers does the database contain?_
1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers?
    1. Which `Country` has the most reviews for its beers? 
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV?
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?
1. Using Graph Algorithms answer **two** of the following questions:
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles? 
    2. <span style="color:red">Which beer has the most similar reviews as the beer `Super Bock Stout` ? </span>
    3. <span style="color:red">Which user is the most influential when it comes to reviews made? </span>
    4. Which beer styles are more central when it comes the amount of beers? 
    5. <span style="color:green">Which beer is the most influential when considering beers are conected by users who review them? </span>
    6. <span style="color:green">Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community? </span> 
    
Notes: 
- We've added some more questions in <span style="color:green">green</span>, so you have a broader choice.
- Questions in <span style="color:red">red</span> have an added dificulty, which will be considered while grading if chosen.
- Consider creating nodes for the STYLES and USERS. 
- For an example on how to perform such CRUD operations, plese use the "load HW1 DB.ipynb" jupyter notebook.
- In case of a tie for the top entity, in terms of metrics outputed from the algorithms, **simply output the first.**

10. If you had to pick 3 beers to recommend using only this database, which would you pick and why?


 Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
#### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

#### Submission      

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of May 16**

#### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

In [ ]:
test = secure_graph.run("""
    CALL gds.graph.project.cypher(
        'example',
        'MATCH (n:COUNTRY) RETURN id(n) AS id',
        'MATCH (n)-[e]-(m) RETURN id(n) AS source, e.weight AS weight, id(m) AS target'
)""").data()

# Note: the name 'full_graph' can only be run once, 
#     then you would need to rename it

### Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "docker stop testneo4j")
#### Let's create a new container for the Homework (notice how it has a different name):
    docker run --name HW1neo4j -p7474:7474 -p7687:7687 -d --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised_address="localhost:7473" --env NEO4J_dbms_connector_http_advertisedaddress="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised_address="localhost:7687" neo4j
    
#### The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the "HW1 Database" file provided in Moodle.
- Copy the path of the "Data" folder of the unziped file, e.g. "C:\Users\nunoa\Desktop\data".
- In your command line (terminal in MacOs) paste the code: "docker cp C:\Users\nunoa\Desktop\data HW1neo4j:/". As you might have notice, you do not have a User called "nunoa", please use the appropriate path that you got from the previous step. Additionally, if your container has a different name than "HW1neo4j", please change it as well.
- Now let's restart our docker container, either in the User Interface (Docker Desktop) or in the command line by typping the command "docker restart HW1neo4j".
- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

In [41]:
import py2neo
from pprint import pprint
username="neo4j"
password="test"
host="localhost"
port="7474"

secure_graph = py2neo.Graph(f"http://{username}:{password}@{host}:{port}")

secure_graph.run("MATCH () RETURN count(*)").data()

[{'count(*)': 9494213}]

#### Understanding the Database

In [42]:
result = secure_graph.run("""
        call db.labels();
""").data()
pprint(result)

[{'label': 'COUNTRIES'},
 {'label': 'CITIES'},
 {'label': 'BREWERIES'},
 {'label': 'BEERS'},
 {'label': 'REVIEWS'}]


In [43]:
result = secure_graph.run("""
        CALL db.relationshipTypes();
""").data()
pprint(result)

[{'relationshipType': 'REVIEWED'},
 {'relationshipType': 'BREWED'},
 {'relationshipType': 'IN'}]


#### 1. Most reviews:  
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

In [44]:
#A
result = secure_graph.run("""

    MATCH p=(br:BEERS)-[rv:REVIEWED]->(rev:REVIEWS)                        
    RETURN br.name as name, count(br.name) as reviews
    ORDER BY reviews DESC 
    LIMIT 1
    
""").data()
result

[{'name': 'IPA', 'reviews': 31387}]

In [45]:
#B
result = secure_graph.run("""

    MATCH m =(rev:REVIEWS)-[rv:REVIEWED]-(br:BEERS)-[:BREWED]-(brw:BREWERIES)
    RETURN brw.name as name, count(brw.name) as reviews
    ORDER BY reviews DESC
    LIMIT 1
    
""").data()
result

[{'name': 'Sierra Nevada Brewing Co.', 'reviews': 175161}]

In [46]:
#C
result = secure_graph.run("""

        MATCH m=(re:REVIEWS)-[:REVIEWED]-(br:BEERS)
        RETURN br.country as name, count(br.country) as reviews 
        ORDER BY reviews DESC 
        LIMIT 1
    
""").data()
result

[{'name': 'US', 'reviews': 7517042}]

#### 2. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?


In [47]:
result = secure_graph.run("""

        MATCH m=(REVIEWS{username:"CTJman"})<-[:REVIEWED]-(br:BEERS)-[r:REVIEWED]->(rev:REVIEWS)
        RETURN rev.username as username, count(br) as reviews  
        ORDER BY reviews DESC 
        LIMIT 5
    
""").data()
result

[{'username': 'acurtis', 'reviews': 1428},
 {'username': 'Texasfan549', 'reviews': 1257},
 {'username': 'kjkinsey', 'reviews': 1205},
 {'username': 'oline73', 'reviews': 1191},
 {'username': 'chippo33', 'reviews': 1161}]

#### 3. Which Portuguese brewery has the most beers?


In [48]:
result = secure_graph.run("""

        MATCH (br:BEERS)-[r:BREWED]-(bre:BREWERIES)
        WHERE br.country = 'PT' 
        RETURN bre.name as name, count(br.name) as beers 
        ORDER BY beers DESC
        LIMIT 1
    
""").data()
result

[{'name': 'Dois Corvos Cervejeira', 'beers': 40}]

#### 4. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


In [49]:
result = secure_graph.run("""

        MATCH m=(re:REVIEWS)-[:REVIEWED]-(br:BEERS)-[:BREWED]-(bre:BREWERIES)-[r:IN]->(cr:CITIES) 
        WHERE br.country = 'PT' AND bre.name = 'Dois Corvos Cervejeira' 
        RETURN br.name as beer, count(br.name) as reviews, bre.name as brewery 
        ORDER BY reviews DESC 
        LIMIT 1
    
""").data()
result

[{'beer': 'Finisterra', 'reviews': 10, 'brewery': 'Dois Corvos Cervejeira'}]

#### 5. On average how many different beer styles does each brewery produce?


In [50]:
result = secure_graph.run("""

    MATCH (brewy:BREWERIES)-[:BREWED]-(br:BEERS)
    WITH brewy.name as brewery_name, count(distinct(br.style)) as styles 
    RETURN round(avg(styles),1) as average
    
""").data()
result

[{'average': 10.7}]

#### 6. Which brewery produces the strongest beers according to ABV?


In [51]:
result = secure_graph.run("""
         MATCH (br:BREWERIES)-[bw:BREWED]->(b:BEERS)
         WHERE toFloat(b.abv)>0
         RETURN DISTINCT(br.name) as Brewery, toFloat(b.abv) as ABV
         ORDER by toFloat(b.abv) DESC 
         LIMIT 1
""").data()
result

[{'Brewery': '1648 Brewing Company Ltd', 'ABV': 100.0}]

#### 7. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?


In [52]:
result = secure_graph.run("""
         MATCH (b:BEERS)-[rw:REVIEWED]->(r:REVIEWS)
         WITH toFloat(r.smell) as aroma, toFloat(r.look) as appearance, b.style as style
         WHERE (aroma > 0 and appearance > 0)
         RETURN DISTINCT style, aroma, appearance, ((appearance+aroma)/2) as score, count(style) as reviews
         ORDER BY score DESC, reviews DESC
         LIMIT 5
""").data()
pprint(result)

[{'appearance': 5.0,
  'aroma': 5.0,
  'reviews': 16774,
  'score': 5.0,
  'style': 'American Imperial Stout'},
 {'appearance': 5.0,
  'aroma': 5.0,
  'reviews': 10268,
  'score': 5.0,
  'style': 'American Imperial IPA'},
 {'appearance': 5.0,
  'aroma': 5.0,
  'reviews': 9209,
  'score': 5.0,
  'style': 'New England IPA'},
 {'appearance': 5.0,
  'aroma': 5.0,
  'reviews': 7620,
  'score': 5.0,
  'style': 'American IPA'},
 {'appearance': 5.0,
  'aroma': 5.0,
  'reviews': 4405,
  'score': 5.0,
  'style': 'Russian Imperial Stout'}]


#### 8. Using Graph Algorithms answer **two** of the following questions:
1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles? 
2. <span style="color:red">Which beer has the most similar reviews as the beer `Super Bock Stout` ? </span>
3. <span style="color:red">Which user is the most influential when it comes to reviews made? </span>
4. Which beer styles are more central when it comes the amount of beers? 
5. <span style="color:green">Which beer is the most influential when considering beers are conected by users who review them? </span>
6. <span style="color:green">Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community? </span> 
    
Notes: 
- We've added some more questions in <span style="color:green">green</span>, so you have a broader choice.
- Questions in <span style="color:red">red</span> have an added dificulty, which will be considered while grading if chosen.
- Consider creating nodes for the STYLES and USERS. 
- For an example on how to perform such CRUD operations, plese use the "load HW1 DB.ipynb" jupyter notebook.
- In case of a tie for the top entity, in terms of metrics outputed from the algorithms, **simply output the first.**

#### 9. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

The main factor for recommending a beer would be its overall score, using the score of taste as a tie breaker. Also, reviews containing positive words such as 'love', 'like', 'good' or 'great' should be taken into consideration

In [53]:
result = secure_graph.run("""

       MATCH (b:BEERS)-[rw:REVIEWED]->(r:REVIEWS)
       WITH b.name as beer, toFloat(r.overall) as overall, toFloat(r.taste) as taste, b.style as style
       WHERE (overall > 0 and taste > 0)
       RETURN DISTINCT beer, style, overall, taste, count(style) as reviews
       ORDER BY overall DESC, taste DESC, reviews DESC
       LIMIT 3 
    
""").data()
result

[{'beer': 'Heady Topper',
  'style': 'New England IPA',
  'overall': 5.0,
  'taste': 5.0,
  'reviews': 2089},
 {'beer': 'Pliny The Elder',
  'style': 'American Imperial IPA',
  'overall': 5.0,
  'taste': 5.0,
  'reviews': 1753},
 {'beer': 'KBS (Kentucky Breakfast Stout)',
  'style': 'American Imperial Stout',
  'overall': 5.0,
  'taste': 5.0,
  'reviews': 1259}]

Group: David Santos (r20181082) and Foazul Islam (m20200750)

The extra materials took too much time to run, and caused serious damage to our computers, so unfortunately, without those materials, we couldn't complete exercise 8